In [3]:
# schwab_1min_scalping.py
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
import ta
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import logging
import os
import psutil
import gc
from dotenv import load_dotenv
import json
import os
from schwab import auth, client
import asyncio

load_dotenv()

False

In [4]:
class ScalpingLogger:
    def __init__(self):
        self.setup_logging()

    def setup_logging(self):
        """Configure efficient logging for 8GB RAM"""
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler('scalping_performance.log', mode='a', encoding='utf-8'),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger('ScalpingSystem')

        # Reduce log file size by rotating
        self.performance_logger = logging.getLogger('Performance')
        perf_handler = logging.FileHandler('scalping_signals.log')
        perf_handler.setFormatter(logging.Formatter('%(asctime)s - %(message)s'))
        self.performance_logger.addHandler(perf_handler)

    def log_memory_usage(self):
        """Log current memory usage"""
        memory = psutil.virtual_memory()
        self.logger.info(f"🧠 Memory: {memory.percent}% used, {memory.available // (1024 ** 3)}GB available")

    def log_scalping_signal(self, signal_data):
        """Log scalping signals efficiently"""
        log_entry = {
            'timestamp': datetime.now().isoformat(),
            'signal': signal_data['signal'],
            'price': signal_data['price'],
            'confidence': signal_data['confidence'],
            'prediction': signal_data['prediction']
        }
        self.performance_logger.info(f"SIGNAL: {json.dumps(log_entry)}")

    def log_data_fetch(self, symbol, price, fetch_time):
        """Log data fetch performance"""
        self.logger.debug(f"📊 {symbol}: ${price:.2f} | Fetch: {fetch_time:.3f}s")

In [5]:
class ScalpingConfig:
    # Trading Configuration
    SYMBOL = "/NQ"  # NASDAQ Futures
    LOOKBACK_PERIOD = 30  # Reduced for 8GB RAM
    UPDATE_INTERVAL = 60  # 1-minute intervals

    # Risk Management
    STOP_LOSS_POINTS = 20
    TAKE_PROFIT_POINTS = 35
    MIN_CONFIDENCE = 0.65

    # AI Model Configuration (Lightweight)
    MODEL_WEIGHTS = {
        'xgboost': 0.6,
        'random_forest': 0.4
    }
class SchwabDataFetcher:
    def __init__(self, easy_client, config, logger):
        self.client = easy_client
        self.config = config
        self.logger = logger
        self.data_buffer = []
        self.last_price = 0

    async def fetch_realtime_quote(self):
        """Fetch real-time quote using easy_client"""
        try:
            start_time = time.time()

            # Get real-time quote
            response = await self.client.get_quotes(self.config.SYMBOL)
            if response.status == 200:
                quote_response_dict = await response.json()
            else:
                self.logger.error(f"API request failed with status code: {response.status}")

            fetch_time = time.time() - start_time

            if quote_response_dict and self.config.SYMBOL in quote_response_dict:
                quote_data = response[self.config.SYMBOL]

                current_data = {
                    'timestamp': datetime.now(),
                    'bid': quote_data.get('bidPrice', 0),
                    'ask': quote_data.get('askPrice', 0),
                    'last': quote_data.get('lastPrice', 0),
                    'volume': quote_data.get('totalVolume', 0),
                    'symbol': self.config.SYMBOL,
                    'fetch_time': fetch_time
                }

                # Calculate mid-price
                current_data['close'] = (current_data['bid'] + current_data['ask']) / 2
                current_data['high'] = max(current_data['bid'], current_data['ask'])
                current_data['low'] = min(current_data['bid'], current_data['ask'])

                self.last_price = current_data['close']

                # Add to buffer
                self.data_buffer.append(current_data)
                if len(self.data_buffer) > self.config.LOOKBACK_PERIOD:
                    self.data_buffer = self.data_buffer[-self.config.LOOKBACK_PERIOD:]
                self.logger.log_data_fetch(self.config.SYMBOL, current_data['close'], fetch_time)
                return current_data

            return None

        except Exception as e:
            self.logger.logger.error(f"❌ Quote fetch error: {e}")
            return None

In [6]:
if __name__ == "__main__":
    # Import and initialize your easy_client
    from schwab import auth  # Adjust import based on your setup


    token_path = 'schwab_token.json'

    try:
    # Initialize easy_client (adjust based on your working code)
        c = auth.easy_client(
            api_key='czjhDYKBQOxEJGoJbRCI9YYoor6z68lP2ustFXCYGT25pDGC',
            app_secret='PacTQniBEOUfCfXa6Z7krvAqLbfFCrrAiRKCpKWxVkOgzsUxo5ar1jNrC91Ngk4w',
            callback_url='https://127.0.0.1:8183',
            token_path=token_path
        )
    except:
        test_response = c.get_quotes('/NQ')
        if test_response:
            print("✅ Schwab connection successful!")
